In [26]:
from logger import Logger
from preparator import Preparator 
from solver import Solver 
import pandas as pd 
from tqdm import tqdm 
from datetime import datetime

import pennylane as qml 
from pennylane import numpy as np
testdev = qml.device('default.qubit', wires = 1)

In [14]:
size = 10
matrix_number = 1000
source_name = 'Traininig_size_{size}.csv'

template = pd.read_csv('data/template.csv')

In [7]:
# Prepare train datasets
log = Logger (log_template = template, log_path = f'data/train/{source_name.format(size = size)}')
exists = log.create_logfile()

Logfile data/train/Traininig_size_10.csv already exists


In [29]:
log_file, id = log._load_logfile()

prep = Preparator()

# for i in tqdm(range(matrix_number), desc = f"Generating random matrices with size {size}"):
    
#     id += 1
#     matrix = prep.rankPreparator(size = size, rank = size)
#     current_datetime = datetime.now()
#     data = { 
#         'Id (int)' : id, 
#         'Valid (int)' : 1, 
#         'Updated (str)' : current_datetime.strftime("%Y-%m-%d %H:%M:%S"),
#         'Matrix (mat)' : matrix,
#         'Size (int)' : size,
#         'Type (str)' : 'Random',
#         'Rank (int)' : size,
#         'Density (flt)' : -1.0,
#         'Stiffness (flt)' : -1.0,
#         'Other (flt)' : -1.0,
#         'Test_solver (str)' : '-',
#         'Test (flt)' : -1.0,
#         'Test_state (vec)' : [],
#         'Test_iterations (int)' : -1,
#         'Test_time (flt)' : -1.0,
#         'Train_solver (str)' : '-',
#         'Preprocessing (str)' : '-',
#         'Postprocessing (str)' : '-',
#         'Train (flt)' : -1.0,
#         'Train_state (vec)' : [],
#         'Train_iterations_1 (int)' : -1,
#         'Train_iterations_2 (int)' : -1,
#         'Train_iterations_3 (int)' : -1,
#         'Train_shots (int)': -1, 
#         'Train_cost (flt)' : -1.0,
#         'Train_time (flt)' : -1.0, 
#         'Flags (str)' : ''
#     }
    
#     log.log(data)



In [37]:

def test (ids : np.ndarray, solver : str = 'bruteforce', log_data : bool = False):

    ''' 
    ids : np.ndarray 
        a list of ids to compute 
    solver : str 
        solver to compute with 
    ''' 
    
    tester = Solver(dev = testdev, ansatz = '')

    for id in tqdm.tqdm(ids, desc = f"Testing matrices with ids {min(ids)} <--> {max(ids)}"):

        data = log.read(id)

        
        def test_solver(Q : np.ndarray, solver : str = 'bruteforce'): 
            ''' 
            tester wrapped
            '''

            match solver: 
                case 'bruteforce':
                    J, h = prep.isingForm(Q)
                    return tester.checkIsing(J, h)
                case 'GW':
                    pass
                case 'Annealing': 
                    pass
            
        Q = data['Matrix (mat)']

        tested_data = data.copy()
        
        test, test_state, test_time = test_solver(Q)
        current_datetime = datetime.now()

        tested_data['Test (flt)'] = test 
        tested_data['Test_state (vec)'] = test_state[0].numpy()
        tested_data['Test_time (flt)'] = test_time
        tested_data['Test_solver (flt)'] = solver
        tested_data['Updated (str)'] = current_datetime.strftime("%Y-%m-%d %H:%M:%S")
 
        if (log_data): log.log(tested_data, byid = True, Id = id)
        else: 
            return tested_data

In [34]:

ids = range (1,4)
number_of_solvers = 3 
poolsize = len(ids)//number_of_solvers

In [ ]:

from multiprocessing import Pool
from functools import partial

if (__name__ == '__main__') and (len(ids) % poolsize == 0): 
    
    solvers = [] 

    ids = np.array(ids) 
    index = 0
    pool_ids = ids.reshape(len(ids) // poolsize, poolsize)
    display(pool_ids)

    args = []
    for i in range (len(pool_ids)):
        if (size > 11):
            args.append((pool_ids[i], 'GW'))
        else: 
            args.append((pool_ids[i], 'bruteforce'))
    print(args)
    with Pool(number_of_solvers) as pool:
        processed_data = pool.starmap(test, args)

for data in processed_data:
    log.log(data = data, byid = True, Id = data['Id (int)'])

# If the error occured you still can try get processed_data 


tensor([[1],
        [2],
        [3]], requires_grad=True)

[(tensor([1], requires_grad=True), 'bruteforce'), (tensor([2], requires_grad=True), 'bruteforce'), (tensor([3], requires_grad=True), 'bruteforce')]
